In [25]:
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
from timeit import default_timer as timer
from scipy import misc
from PIL import Image
import numpy as np
import random
import glob
import math
import cv2 
import os

In [9]:
'''
Decorator to allow other functions to be exectue after one another
'''
def run_after(f_after):
    def wrapper(f):
        def wrapped(*args, **kwargs):
            ret = f(*args, **kwargs)
            f_after()
            return ret
        return wrapped
    return wrapper

overallTimeElapsed = 0

In [10]:
'''
cleaner:
  - Cleans output directory by removing .DS_Store file and last frame.
'''
def cleaner():
    if(os.path.isfile('./clean/.DS_Store')):
        os.remove('./clean/.DS_Store')
    image_list = []
    for filename in glob.glob('./clean/*.jpg'):
        image_list.append(filename)
    image_list.sort()
    le = image_list[-1]
    os.remove('%s' %le)
    print('Clean up completed ✓')

In [13]:
'''
framer:
  - Reads a video using OpenCC
  - Sets image capture to 24 frames per second
  - Reads & Outputs the images into the given path
  - calls cleaner function after function is finished
'''
@run_after(cleaner)
def framer():
    pathIn = './video.mp4'
    pathOut = './clean/'
    if not os.path.exists('./clean'):
      os.makedirs('./clean')
    count = 1
    vidcap = cv2.VideoCapture(pathIn)
    success,image = vidcap.read()
    success = True
    while success:
      vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*45))  
      success,image = vidcap.read()
      print ('Frame ', count, ' Processed.')
      cv2.imwrite( pathOut + "%07d_clean_image.jpg" % count, image)  
      count += 1

In [14]:
'''
isPerfSquare:
    -   Take in a 3D array
    -   Compares the length of the rows vs columns
    -   returns a boolean 
'''                       
def isPerfSquare(arr):
    row = len(arr) #rows
    col = len(arr[0]) #columns
    # print("Number of rows: ", row,'\n')
    # print("Number of columns: ", col,'\n')
    rxc = row - col
    if(rxc != 0):
        return(False)
    return (True)

In [15]:
'''
makePerfSquare : 
    -   Takes in a 3D array
    -   will make the array a perfect square by taking 
        either the row or column and making that the length
        of both sides. 
    -   return a new perfect square array. Padded with zeros to make 
        the perfect square matric possible
'''
def makePerfSquare(arr):
    row = len(arr) #rows
    col = len(arr[0]) #columns
    if(row >  col):
        newarr = np.zeros((row, row, 3), dtype = int)
        for i in range(0, col):
            for j in range(0,col):
                for k in range(0,3):
                    newarr[i][j][k] = arr[i][j][k] 
        return newarr
    else:
        newarr = np.zeros((col, col, 3), dtype = int)        
        for i in range(0, row):
            for j in range(0,col):
                for k in range(0,3):
                    newarr[i][j][k] = arr[i][j][k] 
        return newarr

In [16]:
'''
matrix_mult:
    -   Receives an array
    -   Creates a cypher array which will be used for matrix multiplication
    -   Saves cypher array into its own list which will later be used for inversion (decryption)
    -   applies matrix multiplication to image array and cypher array.
    -   returns new encrypted image array 
'''
count = 0
cypher_list = []
def matrix_mult(arr):
    start = timer()
    global count
    global overallTimeElapsed

    cypher = np.random.random(arr.shape) # Cypher matrix of arr shape
    cypher_list.append(cypher) # Appends cypher to cypher list for matrix inversion
    result = np.zeros(arr.shape) # Cypher matrix of arr shape

    a2 = arr.reshape(-1, arr.shape[1])
    c2 = cypher.reshape(-1, cypher.shape[1])

    # Matrix multiplication
    res = np.dot(a2, c2.T)

    # Reshape 2D matrix mutl into 3d Array
    res3d = res.reshape(len(a2[0]),len(a2), 3)

    # Converts matrix back into an image, then saves the images
    encrypted_image = Image.fromarray(res3d, 'RGB')    
    encrypted_image.save('./dirty/%07d_encrypted_image.jpg' %count)
    end = timer()
    timeElapsed = end - start
    overallTimeElapsed += timeElapsed 
    print("Frame", count, "Encrypted in %.3f" %timeElapsed)

    # Counter for images that have been processed
    count += 1

In [27]:
def encrypt():
  image_list = []
  if not os.path.exists('./dirty'):
    os.makedirs('./dirty')
  for filename in glob.glob('clean/*.jpg'):
      image_list.append(filename)
  image_list.sort()

  # Read all images from image_list and encrypts them
  for image in image_list:
    img = misc.imread(image)    
    isSquare = isPerfSquare(img)
    if(isSquare):
        print('Its a square.') 
    else:  
        sqrArray = makePerfSquare(img)
        matrix_mult(sqrArray)

In [28]:
'''
maker:
  - Todo
'''
def video_maker(images, outimg=None, fps=24, size=None, is_color=True, format="mp4v"):
    global overallTimeElapsed
    print("Overall Encryption time: %.2f" %overallTimeElapsed)
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    outvid = './dirty_video.mp4'
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    print('Video created.')
    return vid

In [20]:
def maker():
  image_list = []
  for filename in glob.glob('dirty/*.jpg'):
      image_list.append(filename)
  image_list.sort()

  video_maker(image_list)

In [29]:
framer()
encrypt()
maker()

Frame  1  Processed.
Frame  2  Processed.
Frame  3  Processed.
Frame  4  Processed.
Frame  5  Processed.
Frame  6  Processed.
Frame  7  Processed.
Frame  8  Processed.
Frame  9  Processed.
Frame  10  Processed.
Frame  11  Processed.
Frame  12  Processed.
Frame  13  Processed.
Frame  14  Processed.
Frame  15  Processed.
Frame  16  Processed.
Frame  17  Processed.
Frame  18  Processed.
Frame  19  Processed.
Frame  20  Processed.
Frame  21  Processed.
Frame  22  Processed.
Frame  23  Processed.
Frame  24  Processed.
Frame  25  Processed.
Frame  26  Processed.
Frame  27  Processed.
Frame  28  Processed.
Frame  29  Processed.
Frame  30  Processed.
Frame  31  Processed.
Frame  32  Processed.
Frame  33  Processed.
Frame  34  Processed.
Frame  35  Processed.
Frame  36  Processed.
Frame  37  Processed.
Frame  38  Processed.
Frame  39  Processed.
Frame  40  Processed.
Frame  41  Processed.
Frame  42  Processed.
Frame  43  Processed.
Frame  44  Processed.
Frame  45  Processed.
Frame  46  Processe

/Users/davidcastaneda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # This is added back by InteractiveShellApp.init_path()


Frame 47 Encrypted in 0.146
Frame 48 Encrypted in 0.122
Frame 49 Encrypted in 0.120
Frame 50 Encrypted in 0.120
Frame 51 Encrypted in 0.120
Frame 52 Encrypted in 0.120
Frame 53 Encrypted in 0.118
Frame 54 Encrypted in 0.117
Frame 55 Encrypted in 0.118
Frame 56 Encrypted in 0.119
Frame 57 Encrypted in 0.117
Frame 58 Encrypted in 0.118
Frame 59 Encrypted in 0.119
Frame 60 Encrypted in 0.118
Frame 61 Encrypted in 0.118
Frame 62 Encrypted in 0.118
Frame 63 Encrypted in 0.129
Frame 64 Encrypted in 0.127
Frame 65 Encrypted in 0.122
Frame 66 Encrypted in 0.118
Frame 67 Encrypted in 0.119
Frame 68 Encrypted in 0.118
Frame 69 Encrypted in 0.118
Frame 70 Encrypted in 0.120
Frame 71 Encrypted in 0.119
Frame 72 Encrypted in 0.119
Frame 73 Encrypted in 0.118
Frame 74 Encrypted in 0.130
Frame 75 Encrypted in 0.133
Frame 76 Encrypted in 0.126
Frame 77 Encrypted in 0.125
Frame 78 Encrypted in 0.121
Frame 79 Encrypted in 0.118
Frame 80 Encrypted in 0.120
Frame 81 Encrypted in 0.119
Frame 82 Encrypted i